# Анализ данных на Python

*Алла Тамбовцева, НИУ ВШЭ*

## Списки vs массивы NumPy

## Часть 1: ещё немного про списки и функцию `zip()`

Пусть у нас есть три списка:

In [1]:
valid = [450, 800, 630, 740, 950]
invalid = [10, 200, 100, 50, 120]
total = [900, 1000, 1200, 890, 2300]

В списке `valid` хранится число действительных бюллетеней на пяти избирательных участках, в списке `invalid` – число недействительных бюллетеней, а в списке `total` – общее число зарегистрированных избирателей. Наша задача – получить список со значениями явки в процентах с точностью до второго знака после запятой. 

Рассмотрим самое прямое, «классическое» решение. В данном случае все списки одинаковой длины, поэтому мы можем пройтись в цикле по индексам элементов и извлечь элементы сразу из трёх списков:

In [2]:
# явка = (действительные + недействительные) / общее число избирателей
# переводим в проценты и округляем до сотых

turnout = []
for i in range(len(valid)):
    res = round((valid[i] + invalid[i]) / total[i] * 100, 2)
    turnout.append(res)
print(turnout)

[51.11, 100.0, 60.83, 88.76, 46.52]


Решение рабочее, но не «питоновское». В Python не рекомендуется использовать цикл `for` в сочетании с `range(len())`, поскольку почти всегда можно найти альтернативу без перебора индексов. В данном случае поможет функция `zip()`. Название этой функции говорящее – она как «молния» на одежде соединяет списки одинаковой длины, образуя пары/тройки/четверки элементов, в зависимости от количества списков:

In [3]:
# list(), так как zip() создает временный скрытый объект
# как range() и map(), например

print(list(zip(valid, invalid)))

[(450, 10), (800, 200), (630, 100), (740, 50), (950, 120)]


In [4]:
print(list(zip(valid, invalid, total)))

[(450, 10, 900), (800, 200, 1000), (630, 100, 1200), (740, 50, 890), (950, 120, 2300)]


Как можно заметить, результат, полученный с `zip()`, можно преобразовать в список, тогда это будет список кортежей. Вложенная структура, для выбора элементов в такой структуре потребуется двойная индексация:

In [5]:
M = list(zip(valid, invalid, total))
M

[(450, 10, 900),
 (800, 200, 1000),
 (630, 100, 1200),
 (740, 50, 890),
 (950, 120, 2300)]

In [6]:
# второй элемент из первого кортежа
print(M[0][1])

# третий элемент из третьего кортежа
print(M[2][2])

# последний элемент из первого кортежа
print(M[0][-1])

10
1200
900


Теперь сделаем перебор по полученному списку троек:

In [7]:
turnout = []

# triple – тройка значений

for triple in zip(valid, invalid, total):
    res = round((triple[0] + triple[1]) / triple[2] * 100, 2)
    turnout.append(res)
print(turnout)

[51.11, 100.0, 60.83, 88.76, 46.52]


Уже получше, но всё равно остались индексы 0, 1, 2. Ещё раз упорстим код, воспользуемся тем, что Python умеет выполнять перебор в цикле `for` сразу по нескольким элементам, если мы укажем их через запятую:

In [8]:
# v – всегда первый элемент в каждой паре
# i – всегда второй элемент в каждой паре
# v – всегда третий элемент в каждой паре

turnout = []
for v, i, t in zip(valid, invalid, total):
    res = round((v + i) / t * 100, 2)
    turnout.append(res)
print(turnout)

[51.11, 100.0, 60.83, 88.76, 46.52]


Такой перебор возможен благодаря тому, что Python поддерживает множественное присваивание (*multiple assignment*): перечни элементов через запятую он сам объединяет в кортеж, а потом понимает, что первый элемент кортежа надо записать в первую переменную, второй – во вторую, и так далее:

In [9]:
a, b = 0, 1
print(a)
print(b)

0
1


Чтобы написать совсем компактный код, который, на этот раз, будет ещё и по скорости исполнения выигрывать, перепишем последний цикл через списковое включение:

In [10]:
turnout = [round((v + i) / t * 100, 2) for v, i, t in zip(valid, invalid, total)] 
print(turnout)

[51.11, 100.0, 60.83, 88.76, 46.52]


Чтобы решить эту задачу ещё более эффективно, имеет смысл перейти к массивам.

## Часть 2: массивы NumPy

Сегодня мы познакомимся с библиотекой NumPy (сокращение от *Numeric Python*), которая часто используется в задачах, связанных с анализом данных и машинным обучением.

Чтобы мы смогли на конкретных примерах увидеть, зачем эта библиотека используется, давайте её импортируем. Если вы уже устанавливали Anaconda, то библиотека NumPy также была установлена на ваш компьютер. Проверим: импортируем библиотеку с сокращенным названием, так часто делают, чтобы не «таскать» за собой в коде длинное название. Сокращение `np` для библиотеки `numpy` – общепринятое, его часто можно увидеть в документации или официальных тьюториалах.

In [11]:
import numpy as np

Если библиотека не найдена, её можно установить через команду `pip install` (восклицательный знак в начале сообщает Jupyter, что далее следует не код на Python, а специальная команда, как будто бы запущенная из консоли, если и так запускаете из консоли, то он не нужен):

In [ ]:
!pip install numpy

Если библиотека уже установлена, устанавливать повторно её не требуется, достаточно просто импортировать. Если библиотека установлена и вы точно знаете, что версия старая, можно запросить принудительную установку более новой версии с помощью опции `--upgrade`.

In [ ]:
!pip install numpy --upgrade

Почему такую установку можно считать «принудительной»? Команда `pip install` действует так: проверяет наличие библиотеки на компьютере или в конкретной рабочей среде, и если хотя бы какая-то версия библиотеки присутствует, установка не производится. Поэтому, добавляя опцию `--upgrade` мы в любом случае инициализируем установку.

Основным объектом NumPy является *Ndarray* – это n-мерный массив (от *n-dimensional array*), структура данных, которая позволяет хранить набор элементов одного типа: либо целые числа, либо числа с плавающей точкой, либо строки, либо логические значения `True` и `False`. Массивы могут быть одномерными, то есть визуально ничем не отличаться от простого списка значений, а могут быть многомерными. 

Как раз с одномерных массивов мы начнём, преобразуем списки `valid`, `invalid` и `total` в массивы, чтобы закончить с задачей из первой части.

In [12]:
Valid = np.array(valid)
Invalid = np.array(invalid)
Total = np.array(total)

Операции на массивах векторизованы – они применяются сразу ко всем элементам массива без циклов и аналогичных конструкций. В нашем случае это означает, что мы можем просто сложить два первых массива:

In [13]:
Valid + Invalid

array([ 460, 1000,  730,  790, 1070])

С обычными списками так бы не получилось, на делении всё бы сломалось, сложение даёт не тот результат:

In [14]:
valid + invalid

[450, 800, 630, 740, 950, 10, 200, 100, 50, 120]

Оператор `+`, применяемый к двум спискам, возвращает новый список – результат их конкатенации (склеивания). Так в Python проявляется перегрузка операторов: для чисел `+` выполняет обычное сложение, для последовательностей – склеивание:

In [15]:
print([2, 4] + [5, 6])
print((2, 0, 6) + (3, 3, 3, 0))
print("abc" + "-" + "cde")

[2, 4, 5, 6]
(2, 0, 6, 3, 3, 3, 0)
abc-cde


С массивами такого не происходит, сложение, как мы видели, выполняется поэлементно. Осталось выполнить остальные операции для нахождения явки:

In [16]:
# плюс метод .round()
# определен на массивах

Turnout = ((Valid + Invalid) / Total * 100).round(2)
print(Turnout)

[ 51.11 100.    60.83  88.76  46.52]


Зачем изучать массивы? Во-первых, как мы убедились, с массивами гораздо приятнее работать, чем со списками, плюс, они занимают меньше памяти. Во-вторых, особенности массивов позволят нам лучше понять, как устроены столбцы в датафреймах (таблицах с данными), с которыми нам предстоит работать дальше.

### Типы данных в массивах и преобразование типов

Чуть раньше мы зафиксировали, что массивы могут состоять только из элементов одного типа. Посмотрим, что это за типы:

In [17]:
# integer: целые числа

numbers = np.array([5, 7, 8, 0, 6])
print(numbers)
print(numbers.dtype)

[5 7 8 0 6]
int64


In [18]:
 # float: вещественные числа
    
decimals = np.array([5.6, 9.3, 2, 9.1])
print(decimals)
print(decimals.dtype)

[5.6 9.3 2.  9.1]
float64


In [19]:
# boolean: логические значения

YN = np.array([True, False])
print(YN)
print(YN.dtype) 

[ True False]
bool


Числа 64 или 32, дописанные в конце названия типа, показывают, сколько битов NumPy готов зарезервировать под хранение значений, учитывая, что сейчас нет жестких ограничений по памяти, как раньше, на это можно не обращать внимания. А вот на что стоит обратить внимание, так это на то, что после `.dtype` нет круглых скобок. Раньше, когда мы дописывали что-то к объекту после точки, это «что-то» было методом (вспомните методы `.append()` и `.lower()` на списках и строках). Здесь `dtype`  – это не метод, а *атрибут* массива, то есть какая-то его фиксированная характеристика.

Три типа рассмотрели, остались строки. Создадим массив со строками:

In [20]:
answers = np.array(["yes", "no", "no answer", "yes"])
print(answers.dtype)

<U9


Получили таинственную запись. Но все просто. Буква `U` здесь означает *Unicode* (в этом формате [кодируются](https://ru.wikipedia.org/wiki/%D0%AE%D0%BD%D0%B8%D0%BA%D0%BE%D0%B4) строки), а 9 – это максимальное число символов в строке внутри массива. Поэтому можем считать это строковым типом, где все строки не длиннее 9 символов. 

В завершение разговора о типах посмотрим, что будет, если мы попытаемся поместить в массив объекты разных типов. Пусть у нас будут названия мячей в квиддиче и число очков, которые они приносят:

In [21]:
balls = np.array(["quaffle", 10, "snitch", 150])
balls

array(['quaffle', '10', 'snitch', '150'], dtype='<U21')

Как и ожидалось, строковый тип оказался сильнее и вытеснил числа. Если это допустимо, можем один тип превратить в другой. Впомним про массив `YN`:

In [22]:
print(YN)

[ True False]


Превратим `True` и `False` в целые числа 1 и 0:

In [23]:
YN2 = YN.astype(int) 
print(YN2)

[1 0]


А теперь в обычные строки:

In [24]:
YN3 = YN.astype(str)
print(YN3)

['True' 'False']


**Важно!** Запомните этот полезный метод `.astype()`, он нам еще очень пригодится, когда будем работать с датафреймами.

### Фильтрация значений по условиям и булевы массивы

Представим себе, что у нас есть массив `points` с числом очков, которые заработала команда за одну игру в квиддич:

In [25]:
points = np.array([150, 0, 20, 0, 30, 20, 0])

Убедимся, что число игроков в команде правильное – должно быть 7 человек. Вызовем атрибут `size`:

In [26]:
print(points.size)  # все ок

7


**NB.** В случае одномерных массивов, как здесь, значение `.size` и результат `len()` совпадает. В случае многомерных массивов будут различия: `.size` всегда хранит общее число элементов, а `len()` будет считать только элементы самого верхнего уровня. Пример матрицы 2 на 2:

In [27]:
# из списка списков
# всего 4 элемента
# но списков внутри всего 2

I = np.array([[0, 1], 
              [1, 2]])

print(I.size)
print(len(I))

4
2


Теперь поинтересуемся, кто из участников набрал больше 0 очков:

In [28]:
points > 0

array([ True, False,  True, False,  True,  True, False])

Неравенство выше было автоматически применено к каждому элементу массива, поэтому мы получили новый массив из `True` и `False`, которые сообщают нам, выполнено ли это условие для конкретного элемента или нет. Как посчитать число игроков, которые заработали больше 0 очков? Посчитать число `True`. А если учесть, что вместо `True` Python видит 1, а вместо `False` – 0? Посчитать сумму всех элементов массива:

In [29]:
(points > 10).sum()

4

А как получить массив, в котором будут только те элементы `points`, которые удовлетворяют некоторому условию? Записать это условие в квадратных скобках, как раньше мы указывали индекс элемента:

In [30]:
points[points > 10]

array([150,  20,  30,  20])

Запись выше означает, что из `points` Python должен выбрать те элементы, где `points > 10` возвращает `True`.

Если условия сложные, то их нужно формулировать с помощью операторов, причём NumPy поддерживает только их символьные версии:

* `&`: одновременное выполнение условий (аналог стандартного `and`);
* `|`: хотя бы одно из условий верно (аналог стандартного `or`);
* `^`: ровно одно верно (формально аналог `xor`, но в Python словесного оператора `xor` нет).

In [31]:
points[(points > 10) & (points < 30)]

array([20, 20])

«Словесные» операторы `and` и `or` здесь не подойдут. Плюс, всегда нужно ставить скобки вокруг каждой части условия, иначе Python начнет «раскручивать» условие со знаков `&` или `|`, что закончится ошибкой:

In [32]:
points[points > 10 & points < 30]  # пытался сопоставить 10 и массив points

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Если нужны индексы элементов, удовлетворяющих условиям, можно воспользоваться функцией `where()`:

In [33]:
np.where(points > 0)

(array([0, 2, 4, 5]),)

**NB.** Функция `where()` всегда проверяет условие для массива по всем измерениям. Здесь оно одно, но результат выдан как будто бы для двух, просто второе место пустое (пустота после запятой). Извлечём индексы в виде одного массива, который при желании можно будет превратить в список:

In [34]:
np.where(points > 0)[0]

array([0, 2, 4, 5])

In [35]:
list(np.where(points > 0)[0])

[0, 2, 4, 5]